### This notebook converts the master network shapefiles exported from CUBE to data structures that can be consumed by Network Wrangler. And saves out the base Network Wrangler files.

In [ ]:
import geopandas as gpd

from pathlib import Path

from network_wrangler import create_scenario
from network_wrangler.roadway.links.create import data_to_links_df
from network_wrangler.roadway.network import RoadwayNetwork

## I/O

In [ ]:
# read the master network shapefile

root_dir = r"Y:\UDOT\Network_Wrangler_Implementation_Phase_1"
data_dir = f"{root_dir}/data"

# the crs is not in the shapefile, so we need to set it manually
# the crs is NAD83 / UTM zone 12N (EPSG:26912)
master_links_gdf = gpd.read_file(f"{data_dir}/interim/master_network/Cachev3_MasterNet_20250328_links.shp")
master_links_gdf = master_links_gdf.set_crs("epsg:26912")
master_nodes_gdf = gpd.read_file(f"{data_dir}/interim/master_network/Cachev3_MasterNet_20250328_nodes.shp")
master_nodes_gdf = master_nodes_gdf.set_crs("epsg:26912")

### Prep CUBE network

In [ ]:
# `PROJECT_NUMBER` got shortened to `PROJECT_NUM` when exporting from CUBE to shapefile, so we need to rename it
master_links_gdf = master_links_gdf.rename(columns={"PROJECT_NUM": "PROJECT_NUMBER"})

In [ ]:
# convert the crs to WGS84
master_links_gdf = master_links_gdf.to_crs("EPSG:4326")
master_nodes_gdf = master_nodes_gdf.to_crs("EPSG:4326")

In [ ]:
# network wrangker requires `model_link_id` integer column in the links dataframe
# we will create this column as a sequential integer starting from 1
# we will also write it out to the modified CUBE network
master_links_gdf['model_link_id'] = range(1, len(master_links_gdf) + 1)

In [ ]:
# fill in the missing columns with default values, 
# if a column is numeric, fill with 0, 
# if a column is string, fill with empty string
for col in master_links_gdf.columns:
    if col not in ['geometry', 'model_link_id']:
        if master_links_gdf[col].dtype == 'object':
            master_links_gdf[col] = master_links_gdf[col].fillna('')
        else:
            master_links_gdf[col] = master_links_gdf[col].fillna(0)

for col in master_nodes_gdf.columns:
    if col not in ['geometry']:
        if master_nodes_gdf[col].dtype == 'object':
            master_nodes_gdf[col] = master_nodes_gdf[col].fillna('')
        else:
            master_nodes_gdf[col] = master_nodes_gdf[col].fillna(0)

In [ ]:
# network wrangker requires `lanes` integer column in the links dataframe
# we will create this column and set it to 0, as the master network does not
master_links_gdf['lanes'] = 0

In [ ]:
# network wrangler expects nodes to have a `model_node_id` attribute
# the master network has `N`, which is the node ID, we will rename it to `model_node_id`
master_nodes_gdf = master_nodes_gdf.rename(columns={'N': 'model_node_id'})

In [ ]:
# this step loads input master network into Network Wrangler's RoadwayNetwork object
wr_links_df = data_to_links_df(master_links_gdf, nodes_df=master_nodes_gdf)
road_net = RoadwayNetwork(nodes_df=master_nodes_gdf, links_df=wr_links_df)

## Create a base scenario

In [ ]:
# create a base scenario without transit
base_scenario = create_scenario(
    base_scenario = {"road_net": road_net}
)

base_scenario = create_scenario(
    base_scenario = base_scenario,
)

## Write out base scenario

In [ ]:
base_scenario.write(
    Path(f"{data_dir}/interim/base_scenario"),
    name = "base",
    roadway_file_format = "geojson",
    roadway_write = True,
    transit_write = False,
    projects_write = True,
    overwrite = True,
)